In [1]:
import cobra
from copy import deepcopy

# Load model
model = cobra.io.read_sbml_model('yeastGEM.xml')

# Correct metabolite ids:
for met in model.metabolites:
    met.id = met.id.replace('__91__', '_')
    met.id = met.id.replace('__93__', '')

model.solver = 'gurobi'


Academic license - for non-commercial use only


In [2]:
print(len(model.reactions))
print(len(model.genes))
print(len(model.metabolites))


3949
1133
2680


In [3]:
print(model.get_metabolite_compartments())

set(['c', 'e', 'g', 'mm', 'm', 'vm', 'ce', 'n', 'p', 'lp', 'v', 'erm', 'gm', 'er'])


In [4]:
SMatrix = cobra.util.array.create_stoichiometric_matrix(model,"dense")
print(len(SMatrix))
print(len(SMatrix[0]))

2680
3949


In [10]:
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0668_e").reactions]) #ethanol
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0565_e").reactions]) #glucose
print([[x.reaction, x.id] for x in model.metabolites.get_by_id("s_0766_e").reactions]) #glycerol
reactionsToConstrain = [list(model.metabolites.get_by_id("s_0668_e").reactions)[1],
                       list(model.metabolites.get_by_id("s_0565_e").reactions)[-1],
                       list(model.metabolites.get_by_id("s_0766_e").reactions)[-1]]


[['s_0668_e <=> s_0666_c', 'r_1147'], ['s_0668_e --> ', 'r_1757']]
[['s_0003_e + s_0805_e --> s_0565_e', 'r_0370'], ['s_0805_e + s_4131_e <=> s_0554_e + s_0565_e', 'r_4420'], ['s_0805_e + s_1466_e --> s_0554_e + s_0565_e', 'r_1024'], ['s_0805_e + s_4140_e --> s_0565_e + s_1106_e', 'r_4400'], ['s_0565_e <=> ', 'r_1714'], ['s_0565_e --> s_0563_c', 'r_1166']]
[['s_0765_c --> s_0766_e', 'r_1172'], ['s_0766_e + s_0796_e --> s_0765_c + s_0794_c', 'r_1171'], ['s_0766_e --> ', 'r_1808']]


In [7]:
model.optimize()
fvaResult = cobra.flux_analysis.variability.flux_variability_analysis(model,reactionsToConstrain,fraction_of_optimum = 0.0)
model.summary()
print(fvaResult)

IN FLUXES          OUT FLUXES      OBJECTIVES
-----------------  --------------  --------------
s_1277_e  2.23     s_0805_e  3.91  r_2111  0.0879
s_0565_e  1        s_0458_e  2.41
s_0796_e  0.789
s_0420_e  0.557
s_1324_e  0.0231
s_1468_e  0.00754
         minimum   maximum
r_1757  0.000000  0.099970
r_1634  0.000000  2.499909
r_1166  0.034104  1.000000
r_1546  0.000000  1.760270


In [8]:
#for 

constrianedModel = cobra.core.model.Model(model)

#inputGlucose = .5
outputEthanol = .05
# outputAcetate = 0.0
# outputLactate = .66
# constraints = [outputEthanol,outputAcetate,inputGlucose,outputLactate]
# for reaction,constraint in zip(reactionsToConstrain[0:len(constraints)],constraints):
#     constrianedModel.reactions.get_by_id(reaction.id).lower_bound = constraint
#     constrianedModel.reactions.get_by_id(reaction.id).upper_bound = constraint

constrianedModel.reactions.get_by_id('r_1757').lower_bound = outputEthanol*(27.0/77.0)
constrianedModel.reactions.get_by_id('r_1757').upper_bound = outputEthanol

constrianedModel.reactions.get_by_id('r_1808').lower_bound = outputEthanol*(27.0/77.0)
constrianedModel.reactions.get_by_id('r_1808').upper_bound = outputEthanol


fvaResult = cobra.flux_analysis.variability.flux_variability_analysis(constrianedModel,reactionsToConstrain,
                                                            fraction_of_optimum=0.0)
print(fvaResult)
constrianedModel.optimize()
constrianedModel.summary()


         minimum   maximum
r_1757  0.017532  0.050000
r_1634  0.000000  2.092463
r_1166  0.196606  1.000000
r_1546  0.000000  1.466968
IN FLUXES          OUT FLUXES        OBJECTIVES
-----------------  ----------------  --------------
s_1277_e  2.19     s_0805_e  3.87    r_2111  0.0731
s_0565_e  1        s_0458_e  2.52
s_0796_e  0.825    s_0668_e  0.0175
s_0420_e  0.463
s_1324_e  0.0193
s_1468_e  0.00628


In [ ]:

geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)